<a href="https://colab.research.google.com/github/benpkxd/Data-Analysis-Bootcamp/blob/main/NYCflights13_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**R_Data_Transformation**


> Description :: 5 basic questions to find insight 🧠

Case Study - NYC Flight13 ✈


*   Q1: Which carriers are in the top3 the most airtime since Feb to Aug ?
*   Q2: Please detail the flights depart to chongqing 
*   Q3: How long from JFK depart to ORD? (Answer with mean)
*   Q4: Please summarize departure_delay in each timezone
*   Q5: Which carrier is the Most , the Least flights , Longest ,Shortest flights





### Import dataset

In [ ]:
install.packages("nycflights13")
install.packages("tidyverse")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
library(tidyverse)
library(nycflights13)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.7      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [ ]:
glimpse(flights)
glimpse(airlines)
glimpse(airports)
glimpse(weather)

Rows: 336,776
Columns: 19
$ year           <int> 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2…
$ month          <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ day            <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ dep_time       <int> 517, 533, 542, 544, 554, 554, 555, 557, 557, 558, 558, …
$ sched_dep_time <int> 515, 529, 540, 545, 600, 558, 600, 600, 600, 600, 600, …
$ dep_delay      <dbl> 2, 4, 2, -1, -6, -4, -5, -3, -3, -2, -2, -2, -2, -2, -1…
$ arr_time       <int> 830, 850, 923, 1004, 812, 740, 913, 709, 838, 753, 849,…
$ sched_arr_time <int> 819, 830, 850, 1022, 837, 728, 854, 723, 846, 745, 851,…
$ arr_delay      <dbl> 11, 20, 33, -18, -25, 12, 19, -14, -8, 8, -2, -3, 7, -1…
$ carrier        <chr> "UA", "UA", "AA", "B6", "DL", "UA", "B6", "EV", "B6", "…
$ flight         <int> 1545, 1714, 1141, 725, 461, 1696, 507, 5708, 79, 301, 4…
$ tailnum        <chr> "N14228", "N24211", "N619AA", "N804JB", "N668DN", "N394…
$ origin      

### Transform data

manipulate NA value

In [ ]:
df <- flights

In [ ]:
#check NA 
df%>%
  apply(MARGIN = 2,function(col) sum(is.na(col))) 
  

year          month            day       dep_time sched_dep_time 
             0              0              0           8255              0 
     dep_delay       arr_time sched_arr_time      arr_delay        carrier 
          8255           8713              0           9430              0 
        flight        tailnum         origin           dest       air_time 
             0           2512              0              0              0 
      distance           hour         minute      time_hour 
             0              0              0              0

In [ ]:
#imputation function
avg_repl_na = function(col){
  replace_na(col,mean(col, na.rm = T))
}

#replace na in air_time column
df$air_time <- df$air_time %>% avg_repl_na
df$dep_delay <- df$dep_delay %>% avg_repl_na

### Q1:  Which carriers are in the top3 the most airtime since Feb to Aug ?

In [ ]:
df_mini<-df %>%
  select(carrier,air_time,month)%>%
  filter(month>=2,month<=8)

df_mini %>% 
  group_by(carrier)%>%
  summarise(total_airtime = sum(air_time))%>%
  arrange(desc(total_airtime))%>%
  head(3)
  

carrier,total_airtime
<chr>,<dbl>
UA,7191137
DL,4910488
B6,4862142


### Q2: Please detail the flights depart to chongqing

In [ ]:
airports%>%
  filter(tzone == "Asia/Chongqing") %>%
  left_join(flights,by = c("faa" = "origin"))

faa,name,lat,lon,alt,tz,dst,tzone,year,month,⋯,arr_delay,carrier,flight,tailnum,dest,air_time,distance,hour,minute,time_hour
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<int>,<int>,⋯,<dbl>,<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dttm>
DVT,Deer Valley Municipal Airport,33.4117,112.457,1478,8,A,Asia/Chongqing,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
MYF,Montgomery Field,32.4759,117.759,17,8,A,Asia/Chongqing,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


### Q3: How long from JFK depart to ORD? (Answer with mean)

In [ ]:
df%>%
  filter(origin == "JFK" & dest == "ORD")%>%
  summarize(durationJFK_to_ORD = mean(hour))

durationJFK_to_ORD
<dbl>
13.96045


### Q4: Please summarize departure_delay in each timezone

In [ ]:
df%>%
  inner_join(airports,by = c("origin" = "faa"))%>%
  group_by(tzone)%>%
  summarize(most_late = max(dep_delay,na.rm = T),
            avg = round(mean(dep_delay,na.rm = T),1),
            most_early = min(dep_delay,na.rm = T))
  
  cat("other timezones are no data")
  

tzone,most_late,avg,most_early
<chr>,<dbl>,<dbl>,<dbl>
America/New_York,1301,12.6,-43


other timezones are no data

### Q5: Which carriers have the Most , the Least flights , Longest ,Shortest flights

In [ ]:
flight_in_each_carrier <- df %>%
  group_by(carrier)%>%
  count(carrier)%>%
  arrange(n)%>%
  left_join(airlines,by = "carrier")
flight_in_each_carrier

carrier,n,name
<chr>,<int>,<chr>
OO,32,SkyWest Airlines Inc.
HA,342,Hawaiian Airlines Inc.
YV,601,Mesa Airlines Inc.
F9,685,Frontier Airlines Inc.
AS,714,Alaska Airlines Inc.
FL,3260,AirTran Airways Corporation
VX,5162,Virgin America
WN,12275,Southwest Airlines Co.
9E,18460,Endeavor Air Inc.


In [ ]:
longest_flight_carrier <- df%>%
  filter(hour == max(hour))%>%
  distinct(carrier)%>%
  left_join(airlines,by = "carrier")
longest_flight_carrier

carrier,name
<chr>,<chr>
B6,JetBlue Airways
UA,United Air Lines Inc.
DL,Delta Air Lines Inc.


In [ ]:
shortest_flight_carrier <- df%>%
  filter(hour == min(hour))%>%
  distinct(carrier)%>%
  left_join(airlines,by = "carrier")
shortest_flight_carrier

carrier,name
<chr>,<chr>
US,US Airways Inc.


In [ ]:
install.packages("glue")
library(glue)

In [ ]:
glue(" Carrier which has The most flight is {flight_in_each_carrier[16,3]} ,The one has least flights is {flight_in_each_carrier[1,3]}")
glue(" Carrier has longest flights is {longest_flight_carrier[,2]} and the one has shortest flight is {shortest_flight_carrier[,2]}")

Carrier which has The most flight is United Air Lines Inc. ,The one has least flights is SkyWest Airlines Inc.

Carrier has longest flights is c("JetBlue Airways", "United Air Lines Inc.", "Delta Air Lines Inc.") and the one has shortest flight is US Airways Inc.

## RpostgreSQL

In [ ]:
install.packages("RPostgreSQL")
library(RPostgreSQL)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: DBI



In [ ]:
con = dbConnect(
  PostgreSQL(),# driver 
  host = "tiny.db.elephantsql.com", #com that run our server
  dbname = "jjcebrax",
  port = 5432, #gate that user access through
  user = "jjcebrax",
  password = "ZkdHpR3d9s9QA3rk7wTeEb07N06tWt6o"
)

In [ ]:
dbListTables(con)
students<-data.frame(
id =  1:4,
name = c("Ben","Bas","Base","Boss"),
grade = c("A","B","C","D")
)
#remove : dbRemoveTable(con,"students_server")
dbWriteTable(con,"students_server",students,row.names=F)

character(0)

[1] TRUE

In [ ]:
dbListTables(con)

[1] "students_server"

In [ ]:
dbGetQuery(con,"SELECT * FROM students_server")

,id,name,grade
,<int>,<chr>,<chr>
1,1,Ben,A
2,2,Bas,B
3,3,Base,C
4,4,Boss,D


In [ ]:
#close connection
dbDisconnect(con)

[1] TRUE